In [ ]:
import time
import random
import matplotlib.pyplot as plt
from data import *
from for_students import *

In [ ]:
items, knapsack_max_capacity = get_big()
print(items)
print(knapsack_max_capacity)

In [ ]:
def roulette_wheel_selection(population, fitness):
    total_fitness = sum(fitness)
    relative_fitness = [f/total_fitness for f in fitness]
    return random.choices(population, weights=relative_fitness, k=2)

def crossover(parent1, parent2):
    crossover_point = len(parent1) // 2
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(child, mutation_rate):
    if random.random() < mutation_rate:
        index = random.randint(0, len(child) - 1)
        child[index] = not child[index]
    return child

def crossover_parents(parents):
    new_population = []
    for i in range(0, len(parents), 2):
        parent1 = parents[i]
        parent2 = parents[i+1]
        child1, child2 = crossover(parent1, parent2)
        new_population.append(child1)
        new_population.append(child2)

    return new_population

def mutate_population(population, mutation_rate):
    return [mutate(child, mutation_rate) for child in population]



In [ ]:
population_size = 100
generations = 200
n_elite = 8
n_selection = population_size - n_elite
mutation_rate = 1

start_time = time.time()
best_solution = None
best_fitness = 0
population_history = []
best_history = []
population = initial_population(len(items), population_size)
for _ in range(generations):
    population_history.append(population)

    new_population = []
    best_individual, best_individual_fitness = population_best(items, knapsack_max_capacity, population)

    fitnesses = [fitness(items, knapsack_max_capacity, individual) for individual in population]
    population_with_fitness = list(zip(population, fitnesses))
    population_with_fitness.sort(key=lambda x: x[1])

    elite_individuals = population_with_fitness[-n_elite:]
    new_population.extend([individual for individual, _ in elite_individuals])

    parent_candidates = population_with_fitness[-n_selection:]
    parents = [individual for individual, _ in parent_candidates]
    children = crossover_parents(parents)
    children = mutate_population(children, mutation_rate)
    new_population.extend(children)

    population = new_population

    if best_individual_fitness > best_fitness:
        best_solution = best_individual
        best_fitness = best_individual_fitness
    best_history.append(best_fitness)

end_time = time.time()
total_time = end_time - start_time
print('Best solution:', list(compress(items['Name'], best_solution)))
print('Best solution value:', best_fitness)
print('Time: ', total_time)

In [ ]:
# plot generations
x = []
y = []
top_best = 10
for i, population in enumerate(population_history):
    plotted_individuals = min(len(population), top_best)
    x.extend([i] * plotted_individuals)
    population_fitnesses = [fitness(items, knapsack_max_capacity, individual) for individual in population]
    population_fitnesses.sort(reverse=True)
    y.extend(population_fitnesses[:plotted_individuals])
plt.scatter(x, y, marker='.')
plt.plot(best_history, 'r')
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.show()